lien pour se documenter :

http://sdz.tdct.org/sdz/le-pattern-decorator-en-python.html

In [1]:
PRIX_BASE = 3.8
PRIX_SALADE = 0.2
PRIX_TOMATES = 0.2
PRIX_OIGNONS = 0.3
PRIX_FRITES = 0.5

class Kebab:
    
    def __init__(self, sauce):
        self.sauce = sauce
        self.base = PRIX_BASE
        self.salade =PRIX_SALADE
        self.tomates = PRIX_TOMATES
        self.oignons = PRIX_OIGNONS
        self.frites = PRIX_FRITES

    @property
    def prix(self):
        return (self.base + self.salade + self.tomates + self.oignons +
                self.frites)

    def __str__(self):
        return 'Kebab'

    def __repr__(self):
        s = 'Kebab ({0}) ;'.format(self.prix)
        for it in self.__dict__.items():
            s +=' {0} ({1}) '.format(*it)
        return s


In [2]:
>>> a = Kebab('ketchup')
>>> print(a)

Kebab


In [3]:
>>> a

Kebab (5.0) ; sauce (ketchup)  base (3.8)  salade (0.2)  tomates (0.2)  oignons (0.3)  frites (0.5) 

## Utilisons l'héritage !

![ heritage entre classe kebab et kebab sans oignons](http://sdz.tdct.org/sdz/medias/imgur.com_ytRli.png)


In [4]:
class KebabSansOignon(Kebab):
    def __init__(self, sauce):
        super().__init__(sauce)
        self.oignons = 0        # Pas d'oignon

    def __str__(self):
        return 'Kebab sans oignon'

In [5]:
>>> a_sans_oignons = KebabSansOignon('ketchup')
>>> print(a_sans_oignons )

Kebab sans oignon


In [6]:
>>> a_sans_oignons 

Kebab (4.7) ; sauce (ketchup)  base (3.8)  salade (0.2)  tomates (0.2)  oignons (0)  frites (0.5) 

```
Bien ! Puisque cette méthode fonctionne jusqu'ici, implémentons une seconde modification : le supplément fromage.
```

In [7]:
PRIX_FROMAGE = 0.5

class KebabSuppFromage(Kebab):
    def __init__(self, sauce):
        super().__init__(sauce)
        self.fromage = PRIX_FROMAGE
        
    @property
    def prix(self):
        return super().prix + self.fromage

    def __str__(self):
        return 'Kebab suppl. fromage'

In [8]:
>>> a_supp_fromage = KebabSuppFromage("ketchup")
>>> print(a_supp_fromage)

Kebab suppl. fromage


In [9]:
>>> a_supp_fromage

Kebab (5.5) ; sauce (ketchup)  base (3.8)  salade (0.2)  tomates (0.2)  oignons (0.3)  frites (0.5)  fromage (0.5) 

```
Et notre architecture donne maintenant :
```

![heritage entre Kebab et les 2 autres classes](http://sdz.tdct.org/sdz/medias/imgur.com_V2lnN.png)

```
Mais ne voyez-vous pas (au moins) un problème se dessiner à l'horizon ?

    Que va t'il se passer lorsque nous voudrons définir un « kebab sans oignon avec supplément fromage » ?

    À quoi ressemblera notre architecture lorsque l'on aura défini toutes les modifications possibles ?

Bien, répondons à ces questions dans l'ordre (mais j'espère que vous y avez réfléchi).
D'abord, si l'on veut définir un « kebab sans oignon avec supplément fromage », on est obligé, si l'on veut suivre notre logique, de dire que c'EST UN « kebab sans oignon » ET UN « kebab avec supplément fromage », ce qui nous donne le schéma suivant :
```

![nouvel architecture](http://sdz.tdct.org/sdz/medias/imgur.com_4rd18.png)

```
Si vous ne le savez pas encore, ceci est une architecture en diamant, et c'est le cas par excellence que l'on cherche à éviter lorsque l'on touche à l'héritage multiple, car cela introduit de très gros risques de bugs et de comportements indéterminés. Dans le cas présent, cela peut encore se gérer, étant donné que nos objets sont très simples, mais le problème reste que notre conception de base nous force à utiliser l'héritage multiple, que chacun d'entre vous devrait considérer comme dangereux.

D'une manière générale, je vous rappelle la règle d'or :
« Si vous PENSEZ avoir besoin de l'héritage multiple, c'est probablement une mauvaise idée. Si vous SAVEZ que vous avez besoin de l'héritage multiple, alors c'est sûrement la seule solution. »

Au cas où vous ne seriez pas encore convaincu que nous avons là un gros problème de conception, essayons de répondre à la seconde question, et envisageons l'intégration des autres personnalisations possibles pour nos kebabs. Personnellement, j'ai eu la flemme de dessiner tout le schéma, mais voici ce que cela donne après quelques ajouts seulement :
```

![](http://sdz.tdct.org/sdz/medias/imgur.com_UHFc1.png)

```
On se retrouve avec un nombre de classes qui croît de manière exponentielle chaque fois que l'on veut introduire une nouvelle personnalisation. Il est évident que ce genre de programme finit très vite par être extrêmement complexe, non maintenable et impossible à faire évoluer.
```

## Une solution

```
Une réflexion légitime serait de se dire « Eh bien ! S'il faut éviter de multiplier les classes, on n'a qu'à gérer les personnalisations directement dans la classe Kebab. ».
```

In [11]:
from collections import namedtuple

PRIX_BASE = 3.0
PRIX_MOUTON = 0.8
PRIX_POULET = 1.0
PRIX_SALADE = 0.2
PRIX_TOMATES = 0.2
PRIX_OIGNONS = 0.3
PRIX_FRITES = 0.5
PRIX_FROMAGE = 0.5

PRIX_MOD_PAIN = 0.5
PRIX_MOD_VIANDE = 1

Ingredient = namedtuple('Ingredient', 'prix quantite')

class Kebab:
    def __init__(self, sauce):
        self.type_sauce = sauce
        self.type_viande = 'mouton'
        self.base = PRIX_BASE

        self._ing = dict()

        self._ing['viande'] = Ingredient(PRIX_MOUTON, 1)
        self._ing['salade'] = Ingredient(PRIX_SALADE, 1)
        self._ing['tomates'] = Ingredient(PRIX_TOMATES, 1)
        self._ing['oignons'] = Ingredient(PRIX_OIGNONS, 1)
        self._ing['frites'] = Ingredient(PRIX_FRITES, 1)
        self._ing['fromage'] = Ingredient(PRIX_FROMAGE, 0)

    @property
    def prix(self):
        return self.base + sum(i.prix*i.quantite for i in self._ing.values())

    def __str__(self):
        s = "Kebab ({0:.2f}) sauce {1}".format(self.prix, self.type_sauce)
        if self.type_viande != "mouton":
            s += ", au {0}".format(self.type_viande)
        for elt in self._ing:
            qte = self._ing[elt].quantite
            if qte > 0:
                s += ", {0}x {1}".format(qte, elt)
            else:
                s += ", sans {0}".format(elt)
        return s

    def sans(self, ingredient):
        if ingredient in self._ing:
            prix, _ = self._ing[ingredient]
            self._ing[ingredient] = Ingredient(prix, 0)

    def supp(self, ingredient):
        if ingredient in self._ing:
            prix, qte = self._ing[ingredient]
            self._ing[ingredient] = Ingredient(prix, qte+1)

    def poulet(self):
        self.type_viande = 'poulet'
        _, qte = self._ing['viande']
        self._ing['viande'] = Ingredient(PRIX_POULET, qte)
    

In [12]:
help(namedtuple)

Help on function namedtuple in module collections:

namedtuple(typename, field_names, *, rename=False, defaults=None, module=None)
    Returns a new subclass of tuple with named fields.
    
    >>> Point = namedtuple('Point', ['x', 'y'])
    >>> Point.__doc__                   # docstring for the new class
    'Point(x, y)'
    >>> p = Point(11, y=22)             # instantiate with positional args or keywords
    >>> p[0] + p[1]                     # indexable like a plain tuple
    33
    >>> x, y = p                        # unpack like a regular tuple
    >>> x, y
    (11, 22)
    >>> p.x + p.y                       # fields also accessible by name
    33
    >>> d = p._asdict()                 # convert to a dictionary
    >>> d['x']
    11
    >>> Point(**d)                      # convert from a dictionary
    Point(x=11, y=22)
    >>> p._replace(x=100)               # _replace() is like str.replace() but targets named fields
    Point(x=100, y=22)



```
Pour résumer l'idée : la classe Kebab encapsule un dictionnaire, qui à chaque nom d'ingrédient associe une quantité ainsi qu'un prix de base. Elle définit 3 méthodes supplémentaires : sans pour ôter un ingrédient, supp pour ajouter un supplément, et, poulet (cette méthode devrait vous mettre la puce à l'oreille), pour changer le type de viande utilisé.

Voici un exemple d'utilisation :
```

In [13]:
>>> a = Kebab("harissa")
>>> print(a)

Kebab (5.00) sauce harissa, 1x viande, 1x salade, 1x tomates, 1x oignons, 1x frites, sans fromage


In [14]:
>>> a.sans('oignons')
>>> print(a)

Kebab (4.70) sauce harissa, 1x viande, 1x salade, 1x tomates, sans oignons, 1x frites, sans fromage


In [15]:
>>> a.supp('fromage')
>>> print(a)

Kebab (5.20) sauce harissa, 1x viande, 1x salade, 1x tomates, sans oignons, 1x frites, 1x fromage


In [16]:
>>> a.supp('viande')
>>> print(a)

Kebab (6.00) sauce harissa, 2x viande, 1x salade, 1x tomates, sans oignons, 1x frites, 1x fromage


In [17]:
>>> a.poulet()
>>> print(a)

Kebab (6.40) sauce harissa, au poulet, 2x viande, 1x salade, 1x tomates, sans oignons, 1x frites, 1x fromage


```
Comme vous le voyez, ça fonctionne plutôt bien : chaque modification se répercute sur le prix du kebab, et tout cela est mis en œuvre grâce à relativement peu de code. Qui plus est, nous n'avons eu qu'une seule classe à implémenter pour arriver à nos fins, ce qui rend notre architecture la plus simple possible.

Il est clair que cette conception est beaucoup plus efficace que la précédente. Cependant, elle souffre encore de plusieurs défauts qui la rendent difficile à faire évoluer. C'est ce que l'on va essayer d'identifier ensemble.
```

## Quand le patron a une nouvelle idée…

```
Imaginons que le patron ait l'idée d'ajouter de nouvelles personnalisations possibles pour un kebab. Par exemple : l'utilisation de pain pita au lieu du pain normal, l'utilisation de viande de dinde au lieu du mouton ou du poulet, ou bien de nouveaux ingrédients en supplément, mettons les « épices du chef ». Que se passerait-il au niveau de notre classe Kebab ?

Voyons un peu.

    Pour ajouter un nouveau type de viande dans les choix possibles, nous serions obligés de créer une nouvelle méthode dinde dans la classe Kebab.

    Pour ajouter un nouvel ingrédient, nous serions obligés d'ajouter un champ "epices du chef" dans le dictionnaire _ing et d'initialiser sa quantité à 0.

    Pour ajouter le choix du type de pain utilisé, nous serions obligés :

        d'ajouter un membre type_pain,

        d'ajouter un champ "pain" dans le dictionnaire _ing,

        d'ajouter une méthode pain_pita.

En bref, notre classe Kebab doit tout savoir des personnalisations possibles : si on veut faire évoluer l'application, nous sommes obligés d'accumuler des modifications dans cette classe, ce qui risque, avec le temps, de la rendre très lourde et illisible. En bref, cela viole le grand principe « Open-Closed » (ou « principe d'ouverture-fermeture ») de la POO.

Pour ceux qui ne le connaîtraient pas encore, je vous rappelle l'énoncé (très zen) de ce principe :

Oooohhhhhhmmm…

« Une classe doit être FERMÉE à la modification, mais OUVERTE à l'extension. »

Amen.

Cela se justifie très simplement : modifier une classe ou une méthode, c'est risquer d'introduire des bugs ou des effets de bord. Seulement une application étant constamment sujette au changement, il faut quand même que nos classes soient réutilisables, même lors d'une évolution de l'application. En bref : nos classes doivent être définies une bonne fois pour toutes, de telle manière qu'il est facile de les faire évoluer sans les modifier, même dans des cas qui n'étaient pas prévus à la base.

Ici, ce n'est pas le cas. Notre nouvelle implémentation, quoique bien meilleure que la précédente, est encore insuffisante : elle marche, mais ne peut pas évoluer facilement. Notre problème de conception est donc un peu plus ardu que prévu… Je pense qu'il est temps maintenant de faire appel à un design pattern.
```

## Decorator à la rescousse !

## Architecture générale du pattern Decorator

![](http://sdz.tdct.org/sdz/medias/imgur.com_poWnY.png)

```
Dans ce schéma, nous avons une classe principale, le Composant abstrait, qui définit l'interface publique de plusieurs ComposantConcrets. Dans notre cas, le Composant se traduirait par une classe Sandwich, dont dérivent en particulier le Kebab, ainsi que d'autres sandwiches "de base", mettons le Cheeseburger, par exemple. En clair, le comportement de base des sandwiches est défini dans une classe de plus haut niveau que le Kebab, et chacune des implémentations concrètes précise le comportement du Sandwich. Jusqu'ici, c'est l'expression la plus pure de l'héritage.

Là où le Decorator se démarque, c'est au niveau de la classe Decorateur. Dans le schéma générique, on définit une interface Decorateur d'où vont dériver de multiples implémentations différentes. Ce qui caractérise ces objets, c'est que non seulement, ils héritent (directement ou non) de l'interface publique de Composant, mais qu'en plus, ils encapsulent un Composant (ou une classe concrète dérivée) afin d'en modifier le comportement ou l'état. Sachant qu'un Decorateur EST UN Composant, cela veut dire qu'un Decorateur peut envelopper indifféremment un ComposantConcret ou un autre Decorateur. Vous en comprenez l'intérêt ?

Dans notre cas, les implémentations concrètes de DecorateurSandwich pourront être Supplement (pour ajouter un ingrédient), ModPain (pour utiliser du pain pita au lieu du pain normal, par exemple), ModViande (pour remplacer le mouton par du poulet), ou RetraitIngredient (pour avoir un kebab « sans oignon »). Étant donné que les décorateurs peuvent s'envelopper les uns les autres, cela va nous permettre de créer toutes les combinaisons de modifications possibles et imaginables, avec très peu d'objets. ;)

Une remarque importante, c'est que les décorateurs ne sont pas vraiment obligés de tous hériter d'une classe Decorateur. En effet : dans le cas où nos objets sont simples (comme ici), on peut sans crainte créer des décorateurs qui héritent directement de Composant. Cependant, pour cette fois, nous allons respecter scrupuleusement le schéma.  
```

## Application du Decorator à notre problème

![Decorateur appliqué au Kebab](http://sdz.tdct.org/sdz/medias/imgur.com_DKpwg.png)

```
Voyons comment ceci peut se traduire en code.

D'abord, la classe Sandwich de base :
```

In [18]:
from collections import namedtuple

Ingredient = namedtuple('Ingredient', 'prix qte')

class Sandwich:
    def __init__(self, sauce):
        self.sauce = sauce
        self._ing = dict()

    @property
    def prix(self):
        return sum(i.prix * i.qte for i in self._ing.values())

    def __repr__(self):
        return "Sandwich sauce {0}".format(self.sauce)

    def __str__(self):
        s = repr(self)
        for key, it in self._ing.items():
            s += "\n {1.qte:>2}x {0:<15}{2:>5.2f} €".format(key, it, it.prix * it.qte)
        s += "\nTotal               {:>5.2f} €".format(self.prix)
        return s

```
Un sandwich est caractérisé par sa sauce, ainsi que la liste de ses ingrédients. La somme des prix × quantite des ingrédients constitue le prix du sandwich. Enfin, deux méthodes utilitaires sont créées, l'une pour décrire le sandwich, et l'autre pour donner le détail du prix.

Voyons maintenant le Kebab :
```

In [19]:
class Kebab(Sandwich):
    def __init__(self, sauce):
        super().__init__(sauce)
        self._ing['base'] = Ingredient(PRIX_BASE, 1)
        self._ing['salade'] = Ingredient(PRIX_SALADE, 1)
        self._ing['tomates'] = Ingredient(PRIX_TOMATES, 1)
        self._ing['oignons'] = Ingredient(PRIX_OIGNONS, 1)
        self._ing['frites'] = Ingredient(PRIX_FRITES, 1)

    def __repr__(self):
        return "Kebab sauce {0}".format(self.sauce)

```
Comme vous le voyez, on se contente de dériver la classe Sandwich et de préciser ses ingrédients (la 'base' représente le pain normal et le mouton).

Commençons par tester cette partie du code :
```

In [20]:
>>> print(Kebab("harissa"))

Kebab sauce harissa
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
  1x oignons         0.30 €
  1x frites          0.50 €
Total                4.20 €


```
Bien. Tout fonctionne.

Créons maintenant notre classe DecorateurSandwich
```

In [21]:
class DecorateurSandwich(Sandwich):
    def __init__(self, sandwich):
        super().__init__(sandwich.sauce)
        self.sandwich = sandwich
        self._ing = sandwich._ing

```
Cette classe remplit le rôle minimal du décorateur

    elle encapsule un Sandwich dont le comportement est destiné à être modifié.

    elle se fait passer pour le Sandwich décoré, de telle manière que le code qui utilisera ce décorateur aura l'impression de traiter directement le Sandwich

C'est pour gérer le second aspect que l'on récupère une référence du membre _ing (le dictionnaire d'ingrédients) du sandwich décoré dans le membre _ing du décorateur : en modifiant les ingrédients du DecorateurSandwich on modifiera directement les ingrédients du Sandwich décoré. 
Essayons maintenant de modéliser un premier décorateur : le retrait d'ingrédients.
```

In [22]:
class RetraitIngredient(DecorateurSandwich):
    def __init__(self, sandwich, ingredient):
        super().__init__(sandwich)
        # suppression de l'ingrédient voulu
        self.retrait = None
        if ingredient in self._ing:
            del self._ing[ingredient]
            self.retrait = ingredient

    def __repr__(self):
        r = repr(self.sandwich)
        # Ajout de la mention "sans X" si un ingrédient 
        # a bien été retiré
        if self.retrait is not None:
            r += ", sans {0}".format(self.retrait)
        return r

In [23]:
>>> a = Kebab("harissa")
>>> print(a)

Kebab sauce harissa
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
  1x oignons         0.30 €
  1x frites          0.50 €
Total                4.20 €


In [24]:
>>> a = RetraitIngredient(a, "oignons") # retrait d'un ingrédient
>>> print(a)

Kebab sauce harissa, sans oignons
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
  1x frites          0.50 €
Total                3.90 €


In [25]:
>>> a = RetraitIngredient(a, "champignons") # retrait d'un ingrédient inexistant
>>> print(a) # aucun effet

Kebab sauce harissa, sans oignons
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
  1x frites          0.50 €
Total                3.90 €


In [26]:
>>> a = RetraitIngredient(a, "frites") # retrait d'un nouvel ingrédient
>>> print(a)

Kebab sauce harissa, sans oignons, sans frites
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
Total                3.40 €


```
On peut d'ailleurs créer quelques fonctions utilitaires pour alléger la syntaxe :

```

In [27]:
def sans_oignon(sandwich):
    return RetraitIngredient(sandwich, 'oignons')

def sans_frite(sandwich):
    return RetraitIngredient(sandwich, 'frites')

def sans_salade(sandwich):
    return RetraitIngredient(sandwich, 'salade')

def sans_tomate(sandwich):
    return RetraitIngredient(sandwich, 'tomates')

```
Ce qui peut nous permettre de faire quelque chose comme :
```

In [28]:
>>> a = sans_frite(sans_oignon(sans_tomate(Kebab("ketchup"))))
>>> print(a)

Kebab sauce ketchup, sans tomates, sans oignons, sans frites
  1x base            3.00 €
  1x salade          0.20 €
Total                3.20 €


In [29]:
class Supplement(DecorateurSandwich):
    def __init__(self, sandwich, ingredient, prix):
        super().__init__(sandwich)
        # ajout de l'ingrédient voulu
        prix, qte = self._ing.get(ingredient, Ingredient(prix, 0))
        self.ajout = ingredient 
        self._ing[ingredient] = Ingredient(prix, qte+1)

    def __repr__(self):
        r = repr(self.sandwich)
        s = ", supplément {0}".format(self.ajout)
        if s not in r:
            r += s
        return r

def supp_fromage(sandwich):
    return Supplement(sandwich, 'fromage', PRIX_FROMAGE)
 
def supp_oignon(sandwich):
    return Supplement(sandwich, 'oignons', PRIX_OIGNONS)

In [30]:
>>> a = supp_fromage(supp_oignon(supp_fromage(Kebab("ketchup"))))
>>> print(a)

Kebab sauce ketchup, supplément fromage, supplément oignons
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
  2x oignons         0.60 €
  1x frites          0.50 €
  2x fromage         1.00 €
Total                5.50 €


```
Il ne reste plus que deux décorateurs à créer : le modificateur de viande, et le modificateur de pain. Ces deux classes sont tellement semblables que l'on pourrait même les réunir en une seule classe. Cependant, utiliser deux classes séparées permet de s'assurer plus simplement que l'on n'effectue pas plusieurs fois une même modification (si le client décide de changer la viande pour "poulet", "oh puis non en fait dinde", il n'a pas à payer deux fois la modification).
```

In [31]:
class ModViande(DecorateurSandwich):
    def __init__(self, sandwich, viande):
        super().__init__(sandwich)
        self.viande = viande
        self.deja_fait = "mod viande" in self._ing
        if not self.deja_fait:
            self._ing["mod viande"] = Ingredient(PRIX_MOD_VIANDE, 1)

    def __repr__(self):
        s = " mod viande {0}".format(self.viande)
        r = repr(self.sandwich).split(',')
        if not self.deja_fait:
            r.append(s)
        else:
            for idx, elt in enumerate(r):
                if "mod viande" in elt:
                    r[idx] = s
                    break
        return ','.join(r)


class ModPain(DecorateurSandwich):
    def __init__(self, sandwich, pain):
        super().__init__(sandwich)
        self.pain = pain
        self.deja_fait = "mod pain" in self._ing
        if not self.deja_fait:
            self._ing["mod pain"] = Ingredient(PRIX_MOD_PAIN, 1)

    def __repr__(self):
        s = " pain {0}".format(self.pain)
        r = repr(self.sandwich).split(',')
        if not self.deja_fait:
            r.append(s)
        else:
            for idx, elt in enumerate(r):
                if "pain" in elt:
                    r[idx] = s
                    break
        return ','.join(r)


def mod_poulet(sandwich):
    return ModViande(sandwich, 'poulet')

def mod_dinde(sandwich):
    return ModViande(sandwich, 'dinde')

def mod_pita(sandwich):
    return ModPain(sandwich, 'pita')

In [32]:
>>> a = mod_dinde(mod_pita(Kebab("ketchup")))
>>> print(a)

Kebab sauce ketchup, pain pita, mod viande dinde
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
  1x oignons         0.30 €
  1x frites          0.50 €
  1x mod pain        0.50 €
  1x mod viande      1.00 €
Total                5.70 €


In [33]:
>>> a = sans_oignon(supp_fromage(mod_poulet(mod_pita(Kebab("mayo/harissa")))))
>>> print(a)

Kebab sauce mayo/harissa, pain pita, mod viande poulet, supplément fromage, sans oignons
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
  1x frites          0.50 €
  1x mod pain        0.50 €
  1x mod viande      1.00 €
  1x fromage         0.50 €
Total                5.90 €


```
 Prenons un peu de recul :

    Toutes les modifications que nous avons prévues sont modélisées par une classe.

    Nous n'avons pas eu besoin de créer trop de classes (une par type de modification).

    Nos classes Sandwich et Kebab n'ont été définies qu'une fois et une seule, 
    et nous n'avons pas eu besoin d'y retoucher.

    La syntaxe des modifications est ridiculement simple à utiliser.

On est bien loin de notre explosion combinatoire du premier jet, 
et de la classe "qui-fait-tout, qui-sait tout" du second essai.
```

## Quand le patron a (encore) une nouvelle idée…

```
    Imaginons que le patron décide d'ajouter un nouvel ingrédient possible en supplément (les épices du chef), comment feriez-vous pour gérer cela dans le code ?

    Comment vous y prendriez-vous pour rendre la sauce personnalisable de manière dynamique (c'est-à-dire APRÈS la création du Sandwich) ?

    Nous n'avons pas encore défini le sandwich Cheeseburger. Comment l'implémenteriez-vous ? Que remarquez-vous, par rapport aux décorateurs que nous avons déjà définis ?

Bien, prenons ces questions dans l'ordre.

Si le patron voulait ajouter un nouvel ingrédient possible en supplément, nous n'aurions pas besoin de modifier une classe : il nous suffirait de connaître son prix, et d'utiliser le décorateur Supplement pour l'ajouter à un sandwich. Pour faire simple, il suffirait en fait de créer une petite fonction :
```

In [34]:
PRIX_EPICES = 0.1
def supp_epices(sandwich):
    return Supplement(sandwich, 'épices du chef', PRIX_EPICES)

In [35]:
>>> a = supp_epices(Kebab("harissa"))
>>> print(a)

Kebab sauce harissa, supplément épices du chef
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
  1x oignons         0.30 €
  1x frites          0.50 €
  1x épices du chef  0.10 €
Total                4.30 €


```
Aucun code à modifier, seulement une petite fonction rajoutée.

Maintenant, si l'on voulait modifier dynamiquement la sauce d'un Kebab, qu'aurions-nous besoin de faire ?
```

In [36]:
>>> a = Kebab('harissa')
>>> print(a)

Kebab sauce harissa
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
  1x oignons         0.30 €
  1x frites          0.50 €
Total                4.20 €


In [37]:
>>> a.sauce = 'ketchup'
>>> print(a)

Kebab sauce ketchup
  1x base            3.00 €
  1x salade          0.20 €
  1x tomates         0.20 €
  1x oignons         0.30 €
  1x frites          0.50 €
Total                4.20 €


In [46]:
PRIX_BASE_CHEESE = 2.0
PRIX_STEAK = 0.5
PRIX_CORNICHONS = 0.2

class Cheeseburger(Sandwich):
    def __init__(self, sauce):
        super().__init__(sauce)
        self._ing['base'] = Ingredient(PRIX_BASE_CHEESE, 1)
        self._ing['steak'] = Ingredient(PRIX_STEAK, 1)
        self._ing['salade'] = Ingredient(PRIX_SALADE, 1)
        self._ing['tomates'] = Ingredient(PRIX_TOMATES, 1)
        self._ing['oignons'] = Ingredient(PRIX_OIGNONS, 1)
        self._ing['fromage'] = Ingredient(PRIX_FROMAGE, 2)
        self._ing['cornichons'] = Ingredient(PRIX_CORNICHONS, 1)

    def __repr__(self):
        return "Cheeseburger sauce {0}".format(self.sauce)

In [47]:
>>> a = sans_oignon(Cheeseburger('ketchup/mayo'))
>>> print(a)

Cheeseburger sauce ketchup/mayo, sans oignons
  1x base            2.00 €
  1x steak           0.50 €
  1x salade          0.20 €
  1x tomates         0.20 €
  2x fromage         1.00 €
  1x cornichons      0.20 €
Total                4.10 €
